<a href="https://colab.research.google.com/github/OleksiiLatypov/Practical_Deep_Learning_with_PyTorch/blob/main/materialist_furniture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'imaterialist-challenge-furniture-2018:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F8220%2F44216%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240510%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240510T194834Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5ed9ca77c4bbeee47c09ce6a7f9767d7c60ba18562e12fc4da7fa4aa38a9651874d4e6c172d87ad94761d7b02c7935080ce7115a3a1738653df1282e614328f77a15edc9d350033ed944f524a2ee389236f550322d2b877c4a7fdae3bbef2f6c4699078758d368cab264608880df1f037f0da4cfbca5c262de6acef63ab1da6849d05b602e989794e7d05ce9aa47dea77feaffd5a85b6f348613227a0403bf3e7cd0a17f90f4cc8ea3a51fdc65dae3c6418cba8a4fb12392347dc4ea5fb32bb98ed732e971f57d1cf73638fae7b3e01ecca04e768b9c533be62215396e40cc1b2ac361d5a78e41a7bc4420e35af04ee304fd449bdc205d936c6a5a9b8c98fc39,imaterialist-furniture-fgvc5:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2463428%2F4174563%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240510%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240510T194834Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4019bbdad841314ab2e69a7dca6c879fa5d95e0b41e31ba7a15388603f66a9ca22b7a29e368f22a0724734ac9e6357e878f303344900ff1c729a5c0cec056af85be987a5fb72f05d6714ca238df628dddd2299b3d72b3e0d5695a384b3a5202014a4713ddd30cde1677d1de3286a17d010ebaa42a2cd0f7fd15a445b64f69d4d9105c3caffea75375a3b6f4c3e38d89493d250711d70de8ceef90c61b7fbcd98504548f13e588626a2235aed2d5859eccc6ad3f1a00827ab549e473645aa5feea173e248d437765677a9a8093db3c0f42f5665fff0690e3b938d71929385ec9e1c5c95a0932d454b2deb941b0b4484bd255d02132705d8a9d08c1f771fa63f56'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os, shutil
import random
import json
import pathlib
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from multiprocessing.pool import ThreadPool as Pool

import matplotlib.pyplot as plt
import seaborn as sns
import urllib3

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, TensorDataset

import torchvision
from torchvision import transforms, models
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder
from torchvision.io import read_image
from torchvision.transforms import ToTensor, Lambda


%matplotlib inline

In [ ]:
class Config:
    # Training
    epochs = 10
    learning_rate = 0.001
    num_classes = 129
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # Data preparation
    train_data_root = '/kaggle/input/imaterialist-challenge-furniture-2018/train.json'
    test_data_root = '/kaggle/input/imaterialist-challenge-furniture-2018/test.json'
    validation_data_root = '/kaggle/input/imaterialist-challenge-furniture-2018/validation.json'
    batch_size = 16
    num_workers = 2
    anotation_file_root = '/kaggle/input/imaterialist-furniture-fgvc5/{}.csv'
    images_path = '/kaggle/input/imaterialist-furniture-fgvc5/images/{}'
    test_anotation_file_root = '/kaggle/working/test_df.csv'

    # Data transformation
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    resize_to = 256
    img_size = 224

In [ ]:
train_data = pd.read_json(Config.train_data_root)
test_data = pd.read_json(Config.test_data_root)
valid_data = pd.read_json(Config.validation_data_root)

print(test_data.size)

In [ ]:
train_data.head()

In [ ]:
def format_dataset(data, test_data=False):
    df = pd.DataFrame()
    if test_data:
        df['image_id'] = data.images.map(lambda x: x['image_id'])
        df['url'] = data.images.map(lambda x: x['url'][0])
    else:
        df['image_id'] = data.annotations.map(lambda x: x['image_id'])
        df['label_id'] = data.annotations.map(lambda x: x['label_id'])
        df['url'] = data.images.map(lambda x: x['url'])
    return df

train_df = format_dataset(train_data)
valid_df = format_dataset(valid_data)
test_df = format_dataset(test_data, test_data=True)

In [ ]:
print(test_df.shape)

In [ ]:
print(test_df.shape)

In [ ]:
test_df.to_csv('test_df.csv', index=False)

In [ ]:
class_count = train_df.groupby(['label_id']).size().sort_values(ascending=False)
class_count

In [ ]:
import seaborn as sns
import pandas as pd

# Grouping the data and getting counts
class_count = train_df.groupby(['label_id']).size().reset_index(name='count')

# Creating the count plot
sns.displot(data=train_df['label_id'])


In [ ]:
def get_transforms(data_subset):
    if data_subset == 'train':
        transformations = transforms.Compose([
            transforms.ToPILImage(),
            transforms.RandomHorizontalFlip(),
            transforms.RandomResizedCrop((Config.img_size, Config.img_size)),
            transforms.ToTensor(),
            transforms.Normalize(Config.mean, Config.std)
        ])
    else:
        transformations = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((Config.img_size, Config.img_size)),
            transforms.ToTensor(),
            transforms.Normalize(Config.mean, Config.std)
        ])
    return transformations

In [ ]:
class CustomImageDataset(TensorDataset):
    def __init__(self, annotations_file, img_dir, transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.shuffle()

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, f'{self.img_labels.iloc[idx, 0]}.jpg')
        image = read_image(img_path)
        image = self.transform(image)
        label = self.img_labels.iloc[idx, 1]
        return image, label

    def shuffle(self):
        self.img_labels = self.img_labels.sample(frac=1).reset_index(drop=True)

In [ ]:
class TestImageDataset(TensorDataset):
    def __init__(self, annotations_file, img_dir, transform=None):
        self.img_ids = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_ids)

    def __getitem__(self, idx):
        image_id = self.img_ids.iloc[idx, 0]
        img_path = os.path.join(self.img_dir, f'{self.img_ids.iloc[idx, 0]}.jpg')
        try:
            image = read_image(img_path)
            image = self.transform(image)
            return image, image_id
        except:
            return [], image_id

In [ ]:
# from PIL import Image
# import requests
# print(train_df.iloc[4,2][0])
# image = requests.get(train_df.iloc[4,2][0],  stream=True)
# print(image.raise_for_status())
# img_open = Image.open(image.raw).convert('RGB')
# img_open

In [ ]:
dfs = {
    'test': test_df,
    'train': train_df,
    'validation': valid_df
}

In [ ]:
dataloaders = {}
image_datasets = {}
for name in ['train', 'validation', 'test']:
    if name == 'test':
        image_datasets[name] = TestImageDataset(Config.test_anotation_file_root,
                                     Config.images_path.format(name),
                                     transform=get_transforms(name))
        dataloaders[name] = DataLoader(image_datasets[name],
                                       shuffle=False,
                                       num_workers=0)
    else:
        image_datasets[name] = CustomImageDataset(Config.anotation_file_root.format(name),
                                     Config.images_path.format(name),
                                     transform=get_transforms(name))
        dataloaders[name] = DataLoader(image_datasets[name],
                                       shuffle=True,
                                       batch_size=Config.batch_size,
                                       num_workers=0)

In [ ]:
def tensor_to_array(img, mean = Config.mean, std = Config.std):
    img = img.numpy()
    np_img = np.transpose(img, (1, 2, 0))
    np_img = np.multiply(np_img, Config.std) + Config.mean
    np_img = np.clip(np_img, 0, 1)

    return np_img

In [ ]:
def show_grid(grid, title=None):
    """
    displays a grid of images
    """
    np_grid = tensor_to_array(grid)

    # display
    plt.figure(figsize=(10, 5))
    plt.axis('off')
    plt.imshow(np_grid)
    if title is not None:
        plt.title(title)

In [ ]:
for name in ['train', 'validation', 'test']:
    for imgs, _ in dataloaders[name]:
        if imgs != []:
            out = make_grid(imgs)
            show_grid(out, f'{name} samples')
            break
        else:
            continue

In [ ]:
model = models.resnet18(pretrained=True).to(Config.device)

for param in model.parameters():
    param.requires_grad = False

output_features = model.fc.in_features

model.fc = nn.Linear(output_features, Config.num_classes).to(Config.device)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=Config.learning_rate, weight_decay=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
def train_model(model, criterion, optimizer, num_epochs=3):
    callbackers = {}
    callbackers['train_loss'] = []
    callbackers['validation_loss'] = []
    callbackers['train_accuracy'] = []
    callbackers['validation_accuracy'] = []
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(Config.device)
                labels = labels.to(Config.device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
            callbackers[f'{phase}_loss'].append(epoch_loss)
            callbackers[f'{phase}_accuracy'].append(epoch_acc)

    return model, callbackers

In [ ]:
model_trained, callbackers = train_model(model, criterion, optimizer, num_epochs=Config.epochs)